In [1]:
import itertools

In [3]:
# Modified from:
# https://pryp.in/blog/15/intersection-and-difference-of-two-rectangles.html
# Generalized to 3D, added a union function, and changed argument order.

class Cuboid:
    def intersection(self, other):
        a, b = self, other
        x1 = max(min(a.x1, a.x2), min(b.x1, b.x2))
        y1 = max(min(a.y1, a.y2), min(b.y1, b.y2))
        z1 = max(min(a.z1, a.z2), min(b.z1, b.z2))
        
        x2 = min(max(a.x1, a.x2), max(b.x1, b.x2))
        y2 = min(max(a.y1, a.y2), max(b.y1, b.y2))
        z2 = min(max(a.z1, a.z2), max(b.z1, b.z2))

        if x1 <= x2 and y1 <= y2 and z1 <= z2:
            return type(self)(x1, x2, y1, y2, z1, z2)
    __and__ = intersection

    def difference(self, other):
        inter = self & other
        
        if not inter:
            yield self
            return
        
        x_split = split((self.x1, self.x2), (other.x1, other.x2))
        y_split = split((self.y1, self.y2), (other.y1, other.y2))
        z_split = split((self.z1, self.z2), (other.z1, other.z2))
                                   
        for (x1, x2), (y1, y2), (z1, z2) in itertools.product(x_split, y_split, z_split):
            rect = type(self)(x1, x2, y1, y2, z1, z2)
            
            if not rect.subset(inter) and rect.subset(self):
                yield rect
    __sub__ = difference
    
    
    def union(self, other):
        # guaranteed no overlap
        inter = self & other
        
        if inter:
            yield inter
        
        yield from self.difference(other)
        yield from other.difference(self)
    __or__ = union
    
    def volume(self):
        return (self.z2 - self.z1 + 1) * (self.y2 - self.y1 + 1) * (self.x2 - self.x1 + 1)

    def __init__(self, x1, x2, y1, y2, z1, z2):
        self.x1, self.x2, self.y1, self.y2, self.z1, self.z2 = x1, x2, y1, y2, z1, z2

    def __iter__(self):
        yield self.x1
        yield self.x2
        yield self.y1
        yield self.y2
        yield self.z1
        yield self.z2

    def __eq__(self, other):
        return isinstance(other, Cuboid) and tuple(self) == tuple(other)
    
    def __ne__(self, other):
        return not (self == other)

    def __repr__(self):
        return type(self).__name__ + repr(tuple(self))
    
    def __hash__(self):
        return hash(self.__repr__())
    
    def subset(self, other):
        return self == self & other
    
    def __copy__(self):
        return type(self)(self.x1, self.x2, self.y1, self.y2, self.z1, self.z2)

In [15]:
cubes = [
    Cuboid(-5, 47, -31, 22, -19, 33),
    Cuboid(-44, 5, -27, 21, -14, 35),
    Cuboid(-49, -1, -11, 42, -10, 38),
    Cuboid(-20, 34, -40, 6, -44, 1),
    Cuboid(-20, 34, -40, 6, -44, 1),
    Cuboid(26, 39, 40, 50, -2, 11),
]

In [16]:
test_cases = itertools.combinations(cubes, 2) 

for (a, b) in test_cases:    
    # none of differences have any overlap
    for (c, d) in itertools.combinations(a.difference(b), 2):
        assert c & d is None
        assert d & c is None
        
    inter = a & b
    inter_volume = inter.volume() if inter else 0
    
    assert a.volume() - inter_volume == sum(c.volume() for c in a.difference(b))
    assert b.volume() - inter_volume == sum(c.volume() for c in b.difference(a))
    
    assert a.volume() + b.volume() - inter_volume == sum(c.volume() for c in a.union(b))
    assert a.volume() + b.volume() - inter_volume == sum(c.volume() for c in b.union(a))